In [ ]:
import os, pickle, pystan, time
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from mne import read_label, read_surface
from scripts.excursion import excursion_mc
from scripts.utilities import assemble_advi_params, tris_to_adj
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
%matplotlib inline

zscore = lambda arr: (arr - arr.mean()) / arr.std()

# Section 1: Preparations
## Load and prepare data

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load surface / label data.
_, tris = read_surface('data/visual/lh.white')
label = read_label('data/visual/lh.Medial_wall.label')

## Load fMRI data.
Y = np.load('data/visual/sub-01_task-visualcontrol_space-fsaverage5.L.psc.npz')['psc']
total_time, total_vert = Y.shape

## Load regressors.
X = np.loadtxt('data/visual/sub-01_task-visualcontrol_events.txt')
if X.ndim == 1: X = X.reshape(-1,1)

Z = np.loadtxt('data/visual/sub-01_task-visualcontrol_motion.txt')
if Z.ndim == 1: Z = Z.reshape(-1,1)
    
'''CURRENT HACK -- WILL BE REMOVED'''
Z = np.apply_along_axis(zscore, 0, Z[:,1:]) 
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Remove triangles with vertices in the medial wall.
ix = np.any(np.apply_along_axis(np.in1d, 0, tris, label.vertices), axis=-1)
tris = tris[np.invert(ix)]

## Make adjacency matrix.
A = tris_to_adj(tris, remap_vertices=True, verbose=False).tocsr()

## Compute degree.
D = np.asarray(A.sum(axis=0)).squeeze()

## Remove vertices in medial wall.
ix = np.in1d(np.arange(total_vert), label.vertices)
Y = Y[:,np.invert(ix)]

## Normalize data.
global_mean = np.mean(Y)
global_var = np.std(Y)
Y = zscore(Y)

## Define metadata.
T, V = Y.shape
T, K = X.shape
T, M = Z.shape

## Load Stan Models

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Mass Univariate Model.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define path.
f = 'stan_models/MassUnivariateAR.pkl'

if not os.path.isfile(f):
    
    ## Compile StanModel.
    MassUnivariate = pystan.StanModel(file=f.replace('pkl','stan'))
    
    ## Dump StanModel to file.
    with open(f, 'wb') as f: pickle.dump(MassUnivariate, f)
        
else:
    
    ## Load StanModel.
    MassUnivariate = pickle.load(open(f,'rb'))
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Unweighted Graph Laplacian (IAR) Model.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define path.
f = 'stan_models/IAR.pkl'

if not os.path.isfile(f):
    
    ## Compile StanModel.
    IAR = pystan.StanModel(file=f.replace('pkl','stan'))
    
    ## Dump StanModel to file.
    with open(f, 'wb') as f: pickle.dump(MassUnivariate, f)
        
else:
    
    ## Load StanModel.
    IAR = pickle.load(open(f,'rb'))

## Section 2: Estimation
### Mass Univariate
The simplest model is the standard model in fMRI: mass univariate. In mass univariate, all regression coefficients are estimated independently of one another. 

For a single subject, we have a set of observations $Y \in \mathbb{R}^{T,V}$, where $T$ is the number of time points and $V$ is the number of voxels. We also have a set of task regressors observations $X \in \mathbb{R}^{T,K}$ and nuisance regressors $Z \in \mathbb{R}^{T,M}$, where $K$ and $M$ are the number of task and nuisance regressors, respectively. We assume that the observed BOLD signal is a linear combination of the weighted sets of task and nuisance regressors:

$$ Y = XB + ZG + \epsilon $$

where $B \in \mathbb{R}^{K,V}$ and $G \in \mathbb{R}^{M,V}$ are the regression weights mapping the task and nuisance regressors, respectively. We assume the errors are normally distributed such that:

$$ \epsilon \sim \mathcal{N}(0, \Sigma) $$

$$ Y \sim \mathcal{N}(XB + ZG, \Sigma) $$

where $\Sigma \in \mathbb{R}^{V,V}$ is the covariance matrix. If we assume *independent and identically distributed (iid)* residuals, then the covariance matrix is equivalent to $\Sigma = \sigma^2 I$. 

Unfortunately fMRI data violates the *iid* assumptions due to temporally correlated residuals as a result of non-neural contributions to the BOLD signal (e.g. respiratory, scanner noise). 

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Prepare data.
AR = 1
data = dict(T=T, V=V, K=K, M=M, AR=AR, Y=Y, X=X, Z=Z)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute MAP estimates.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## OLS estimates used as starting point for 
## MAP sampling.

st = time.time()
MAP = MassUnivariate.optimizing(data=data, seed=47404)
print('MAP elapsed time: %0.1f s' %(time.time() - st))

In [ ]:
from scripts.utilities import phi_approx
rho = phi_approx(MAP['rho_pr'])
plt.hist(rho.squeeze())

In [ ]:
B = MAP['B']
B *= global_var
plt.hist(B.squeeze())

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute VB estimates.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## MAP estimates used as starting point for
## ADVI sampling.

st = time.time()
VB = MassUnivariate.vb(data=data, init=MAP, output_samples=1000, seed=47404)
print('VB elapsed time: %0.1f s' %(time.time() - st))

## Reassemble outputs.
param_names = ['B','G','sigma','rho']
dim = [[V,K],[V,M],[1],[V,AR]]
params = assemble_advi_params(VB['sampler_params'], param_names, dim)

## Dump to file.
f = 'data/visual/sub-01_MassUnivariate.L.psc.pkl'
with open(f, 'wb') as f: pickle.dump(params, f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute excursion set and overlay.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
## Compute excursion set.
B = params['B'].squeeze()
E = excursion_mc(B, u=1, tail=1, alpha=0.05)
    
## Mask percent signal change by excursion set.
psc = np.median(B, axis=-1)
psc = np.where(np.in1d(np.arange(V), E), psc, 0)

## Save overlay.
overlay = np.zeros(total_vert)
overlay[np.invert(ix)] = psc
overlay = nib.Nifti1Image(overlay.reshape(-1,1,1,1), np.identity(4))
nib.save(overlay, 'data/visual/sub-01_MassUnivariate.L.psc.nii.gz')

### Unweighted Graph Laplacian (IAR)

The simplest model of spatial dependence we will deal with is a simple smoothing prior on the regression weights $\vec{w}$ of length $V$:

$$ p(\vec{w} \mid \tau^{-1}, L^{-1}) = \mathcal{N}(0, \tau^{-1} L^{-1} ) $$

where $L$ is the **unweighted graph laplacian (UGL)**, defined as:

$$ L = D - A $$

where $D$ is a $VxV$ diagonal matrix where values $d_{i,i}$ is the *degree* of the corresponding vertex, and $A$ is the adjacency matrix of the mesh. Concretely, the UGL matrix can be defined as:

$$ L_{i,j} = \begin{cases} d_{i} & \text{if} \ i=j \\ -1 & \text{if} \ adj(i,j) \\ 0 & \text{otherwise} \end{cases} $$

The UGL matrix can be interpreted as a differencing matrix; in other words, the values of $\vec{w}$ are penalized for their differences from their neighbors. 

The UGL is worth considering as it is popular and the default prior for Bayesian analysis in SPM.

Formally, we can express the prior above as:

$$ p(\vec{w} \mid \tau^{-1}, L^{-1}) = \frac{1}{\sqrt{2 \pi^n \left| \tau^{-1} L^{-1} \right|}} \text{exp}\left(-\frac{1}{2}\vec{w}^T (\tau L) \vec{w}\right)$$

We will take the log-likelihood:

$$ \text{log} \ p(\vec{w} \mid \tau^{-1}, L^{-1})= -\frac{1}{2} \left( n\text{log}(2 \pi) + \text{log}\left(\left| \tau^{-1} L^{-1} \right|\right) + \vec{w}^T (\tau L) \vec{w} \right) $$

Drop constant:

$$ =  -\frac{1}{2}\text{log}\left(\left| \tau^{-1} L^{-1} \right|\right) -\frac{1}{2} \vec{w}^T (\tau L) \vec{w} $$

Factor out $\tau$:

$$ = -\frac{1}{2}\text{log}\left(\tau^{-n} \left| L^{-1} \right|\right) -\frac{1}{2} \vec{w}^T (\tau L) \vec{w} $$

$$ = \frac{n}{2}\text{log}(\tau) - \frac{1}{2}\text{log}\left( \left| L^{-1} \right|\right) -\frac{1}{2} \vec{w}^T (\tau L) \vec{w} $$

$\tau L$ is sparse, so matrix multiplication will be quick. That said, its inverse determinant is two $O(n^3)$ computations. However, [Jin, Carlin, and Banerjee (2005)](https://www.ncbi.nlm.nih.gov/pubmed/16401268) show that:

$$ \mid L \mid = \mid D - A \mid \propto \prod_{i=1}^{n}(1 - \lambda_i) $$

where $\lambda_1, ..., \lambda_i$ are the eigenvalues of $D^{-\frac{1}{2}}WD^{-\frac{1}{2}}$, which can be computed ahead of time and passed in as data. Finally, we know that $\text{det}(A^{-1}) = \frac{1}{\text{det}(A)}$, therefore:

$$ = \frac{n}{2}\text{log}(\tau) - \frac{1}{2}\text{log}\left( \frac{1}{\prod_{i=1}^{n}(1 - \lambda_i)} \right) -\frac{1}{2} \vec{w}^T (\tau L) \vec{w} $$

Notably, the product of the eigenvalues just becomes another constant and can therefore be dropped too. So we end up with:

$$ = \frac{n}{2}\text{log}(\tau) -\frac{1}{2} \vec{w}^T (\tau L) \vec{w} $$

Very sparse!

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Prepare data.
AR = 1
data = dict(T=T, V=V, K=K, M=M, AR=AR, nz=A.data.size, Y=Y, X=X, Z=Z, D=D, 
            Aw=A.data, Av=A.indices+1, Au=A.indptr+1)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute MAP estimates.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## MAP estimates used as starting point for
## ADVI sampling.

st = time.time()
MAP = MassUnivariate.optimizing(data=data, seed=47404)
print('MAP elapsed time: %0.1f s' %(time.time() - st))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute VB estimates.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

st = time.time()
VB = MassUnivariate.vb(data=data, init=MAP, output_samples=1000, seed=47404)
print('VB elapsed time: %0.1f s' %(time.time() - st))

## Reassemble outputs.
param_names = ['B','G','sigma','rho']
dim = [[V,K],[V,M],[1],[V,AR]]
params = assemble_advi_params(VB['sampler_params'], param_names, dim)

## Dump to file.
f = 'data/visual/sub-01_IAR.L.psc.pkl'
with open(f, 'wb') as f: pickle.dump(params, f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute excursion set and overlay.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
## Compute excursion set.
B = params['B'].squeeze()
E = excursion_mc(B, u=1, tail=1, alpha=0.05)
    
## Mask percent signal change by excursion set.
psc = np.median(B, axis=-1)
psc = np.where(np.in1d(np.arange(V), E), psc, 0)

## Save overlay.
overlay = np.zeros(total_vert)
overlay[np.invert(ix)] = psc
overlay = nib.Nifti1Image(overlay.reshape(-1,1,1,1), np.identity(4))
nib.save(overlay, 'data/visual/sub-01_IAR.L.psc.nii.gz')